### Extracted 10 year historical price data for the top 25 SP500 stocks (ranked by MarketCap -- according to Investopedia) using Yahoo Finance API through RapidAPI 

#### columns --> date, date_utc, open, high, low, close, adjclose, volume, symbol 

In [1]:
'''
Python Script that pulls 10 years of historical price data for the top 25 SP500 Stocks
'''

# import libraries
import requests
import json 
from dotenv import load_dotenv
import os, glob 
import pandas as pd

# list of top 25 SP500 stocks 
top_25_sp500_stocks = ['AAPL', 'MSFT', 'AMZN', 'TSLA', 'GOOGL', 
                       'GOOG', 'BRK-B', 'UNH', 'JNJ', 'XOM', 
                       'JPM', 'META', 'V', 'PG', 'NVDA', 'HD',
                       'CVX', 'LLY', 'MA', 'ABBV', 'PFE', 'MRK', 
                       'PEP', 'BAC', 'KO']

# loop through list of stocks and pull data from API 
for stock in top_25_sp500_stocks: 

    url = "https://yahoo-finance15.p.rapidapi.com/api/yahoo/hi/history/" + stock + "/1d"
    
    querystring = {"diffandsplits":"false"}

    load_dotenv()

    X_RapidAPI_Key = os.getenv("X-RapidAPI-Key")

    headers = {
	    "X-RapidAPI-Key": X_RapidAPI_Key,
	    "X-RapidAPI-Host": "yahoo-finance15.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    # store API data 
    data = response.json()

    # store stock symbol 
    symbol_df = pd.json_normalize(data['meta'])

    # pull stock details such as open, low, high, close, adjclose, and volume 
    stock_details = []
    for key, val in data['items'].items():
        stock_details.append(val)

    # store stock details 
    stock_details_df = pd.json_normalize(stock_details)

    # add stock symbol into dataframe
    stock_details_df['symbol'] = symbol_df['symbol']
    stock_details_df['symbol'].fillna(stock, inplace=True)
   
    # create csv
    stock_details_df.to_csv('../Resources/data_' + stock + '_historical_prices.csv', index = False)

In [2]:
# merge top 25 SP500 stock data csv files in to a single csv for data analysis 
path = "../Resources/"

all_files = glob.glob(os.path.join(path, "data_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv( "../Resources/merged_top25_sp500_stock_data.csv")

In [1]:
import pandas as pd

# review merged stock data 
merged_top25_sp500_stocks_df = pd.read_csv(
    '../Resources/merged_top25_sp500_stock_data.csv',
    index_col='date', 
    infer_datetime_format=True,
    parse_dates=True
)

# drop un-necessary columns
merged_top25_sp500_stocks_df = merged_top25_sp500_stocks_df.loc[:, ~merged_top25_sp500_stocks_df.columns.str.contains('^Unnamed')]
merged_top25_sp500_stocks_df.drop(['date_utc'], axis=1, inplace=True)

merged_top25_sp500_stocks_df.info()

display(merged_top25_sp500_stocks_df.head())
display(merged_top25_sp500_stocks_df.tail())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 62883 entries, 2012-10-31 to 2022-10-28
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   open      62883 non-null  float64
 1   high      62883 non-null  float64
 2   low       62883 non-null  float64
 3   close     62883 non-null  float64
 4   volume    62883 non-null  int64  
 5   adjclose  62883 non-null  float64
 6   symbol    62883 non-null  object 
dtypes: float64(5), int64(1), object(1)
memory usage: 3.8+ MB


,open,high,low,close,volume,adjclose,symbol
date,,,,,,,
2012-10-31,21.25,21.50,20.99,21.26,510003200,18.23,AAPL
2012-11-01,21.36,21.54,21.22,21.31,361298000,18.27,AAPL
2012-11-02,21.28,21.32,20.53,20.60,599373600,17.66,AAPL
2012-11-05,20.84,20.99,20.63,20.88,529135600,17.90,AAPL
2012-11-06,21.08,21.10,20.72,20.82,374917200,17.85,AAPL


,open,high,low,close,volume,adjclose,symbol
date,,,,,,,
2022-10-24,105.66,107.68,105.12,106.60,17024200,106.60,XOM
2022-10-25,106.41,107.33,105.07,105.88,15719800,105.88,XOM
2022-10-26,106.26,107.83,106.00,107.14,17383800,107.14,XOM
2022-10-27,108.41,109.58,107.42,107.55,18214000,107.55,XOM
2022-10-28,109.46,111.21,108.12,110.70,28158200,110.70,XOM
